In [133]:
import numpy as np 
import pandas as pd 
from scipy import stats
# import sklearn as sk 
import matplotlib as plot
import requests as req 
import json
import unidecode
from nba_api.stats.endpoints import leagueseasonmatchups

In [119]:
team_map = {"ATL" : "Atlanta Hawks", "BKN" : "Brooklyn Nets", "BOS": "Boston Celtics", "CHA" : "Charlotte Hornets", "CHI" : "Chicago Bulls", "CLE" : "Cleveland Cavaliers", "DAL" : "Dallas Mavericks", "DEN" : "Denver Nuggets", "DET" : "Detroit Pistons", "GSW" : "Golden State Warriors", "HOU" : "Houston Rockets", "IND" : "Indiana Pacers", "LAC" : "Los Angeles Clippers", "LAL" : "Los Angeles Lakers", "MEM" : "Memphis Grizzlies", "MIA" : "Miami Heat", "MIL" : "Milwaukee Bucks", "MIN" : "Minnesota Timberwolves","NOP" : "New Orleans Pelicans", "NYK" : "New York Knicks", "OKC" : "Oklahoma City Thunder", "ORL" : "Orlando Magic", "PHI" : "Philadelphia 76ers", "PHX" : "Phoenix Suns", "POR" : "Portland Trail Blazers", "SAC" : "Sacramento Kings", "SAS" : "San Antonio Spurs", "TOR" : "Toronto Raptors", "UTA" : "Utah Jazz", "WAS" : "Washington Wizards"}

df_team = pd.read_csv('../data/teams_stats/2019-20_nba_teams.csv')
df_player = pd.read_csv('../data/player_stats/2019-20_nba_players_new.csv')

df_playerid = pd.read_csv('../data/nba_api/all_players.csv')
df_playerid = df_playerid[['id', 'full_name']]

def stripPeriods(s):
    return s.replace('.', '')

df_playerid['full_name'] = df_playerid['full_name'].apply(stripPeriods)


In [120]:
df_playerid.head()

,id,full_name
0,76001,Alaa Abdelnaby
1,76002,Zaid Abdul-Aziz
2,76003,Kareem Abdul-Jabbar
3,51,Mahmoud Abdul-Rauf
4,1505,Tariq Abdul-Wahad


In [121]:
df_player.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Steven Adams/adamsst01,C,26,OKC,63,63,1680,283,478,...,0.582,207,376,583,146,51,67,94,122,684
1,2,Bam Adebayo/adebaba01,PF,22,MIA,72,72,2417,440,790,...,0.691,176,559,735,368,82,93,204,182,1146
2,3,LaMarcus Aldridge/aldrila01,C,34,SAS,53,53,1754,391,793,...,0.827,103,289,392,129,36,87,74,128,1001
3,4,Kyle Alexander/alexaky01,C,23,MIA,2,0,13,1,2,...,NaN,2,1,3,0,0,0,1,1,2
4,5,Nickeil Alexander-Walker/alexani01,SG,21,NOP,47,1,591,98,266,...,0.676,9,75,84,89,17,8,54,57,267


In [122]:
def stripAsterisk(s):
    if s[-1] == '*':
        return s[0:-1]
    else:
        return s

df_team['Team'] = df_team['Team'].apply(stripAsterisk)
df_team = df_team.dropna()
df_team.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PTSA
0,1.0,Dallas Mavericks,75,18175,3124,6772,0.461,1136,3095,0.367,...,791,2729,3520,1849,460,363,953,1462,8776,8405
1,2.0,Milwaukee Bucks,73,17595,3160,6638,0.476,1007,2840,0.355,...,691,3083,3774,1889,526,429,1102,1431,8663,7927
2,3.0,Portland Trail Blazers,74,17835,3122,6749,0.463,952,2525,0.377,...,754,2599,3353,1525,465,451,946,1606,8508,8593
3,4.0,Houston Rockets,72,17380,2936,6512,0.451,1126,3261,0.345,...,704,2484,3188,1557,627,371,1057,1566,8482,8269
4,5.0,Los Angeles Clippers,72,17380,2992,6425,0.466,895,2410,0.371,...,767,2664,3431,1708,511,338,1051,1594,8377,7913


In [123]:
def nameOnly(s):
    new_s = s.split('/')
    return new_s[0]

df_player['Player'] = df_player['Player'].apply(nameOnly)
df_player.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Steven Adams,C,26,OKC,63,63,1680,283,478,...,0.582,207,376,583,146,51,67,94,122,684
1,2,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,...,0.691,176,559,735,368,82,93,204,182,1146
2,3,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,...,0.827,103,289,392,129,36,87,74,128,1001
3,4,Kyle Alexander,C,23,MIA,2,0,13,1,2,...,NaN,2,1,3,0,0,0,1,1,2
4,5,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,...,0.676,9,75,84,89,17,8,54,57,267


In [132]:
df_player = df_player.drop_duplicates(subset="Player", keep="last")

df_player['ID'] = 0

for index, row in df_player.iterrows():
    player = unidecode.unidecode(row['Player']).replace('.', '')
    result = df_playerid[df_playerid['full_name'].str.match(player)]
    if not result.empty:
        df_player.at[index,'ID'] = int(result['id'])

print(len(df_player))

df_player = df_player[df_player['ID'] != 0]

print(len(df_player))

df_player.to_csv('suffering.csv')


518
518


In [134]:
# ---------------------------------
#  WEB SCRAPE TO GET DATA TO CALCULATE DEFENSIVE RATING
# -------------------------------------
lsm = leagueseasonmatchups.LeagueSeasonMatchups(league_id = '00', per_mode_simple = 'Totals', season = '2019-20', season_type_playoffs = 'Regular Season', def_player_id_nullable = '2544').get_data_frames()

lsm[0]

AttributeError: 'list' object has no attribute 'head'

In [126]:
# higher ortg is better
def assignORtg(row):
    FGA = row['FGA']
    ORB = row['ORB']
    TOV = row['TOV']
    FTA = row['FTA']

    TotalP = FGA - ORB + TOV + (0.4 * FTA)

    PTS = row['PTS']

    return (PTS/TotalP) * 100

# Lower drtg is better
def assignDRtg(row):
    FGA = row['FGA']
    ORB = row['ORB']
    TOV = row['TOV']
    FTA = row['FTA']

    TotalP = FGA - ORB + TOV + (0.4 * FTA)

    PTSA = row['PTSA']

    return (PTSA/TotalP) * 100

# def assignNRtg(row):
#     return row['ORTG'] - row['DRTG']


df_team['ORTG'] = df_team.apply(assignORtg, axis=1)
df_team['DRTG'] = df_team.apply(assignDRtg, axis=1)
# df_team['NRTG'] = df_team.apply(assignNRtg, axis=1)
df_team.sort_values(by=['ORTG'])

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,TRB,AST,STL,BLK,TOV,PF,PTS,PTSA,ORTG,DRTG
28,29.0,Golden State Warriors,65,15725,2510,5730,0.438,678,2032,0.334,...,2785,1663,534,299,969,1304,6912,7478,103.839913,112.343008
25,26.0,New York Knicks,66,15965,2638,5896,0.447,631,1872,0.337,...,3069,1456,504,313,946,1464,6983,7409,104.739763,111.129444
29,30.0,Charlotte Hornets,65,15750,2425,5586,0.434,785,2231,0.352,...,2781,1549,428,268,949,1223,6687,7126,104.772499,111.650790
27,28.0,Chicago Bulls,65,15675,2573,5762,0.447,793,2282,0.348,...,2721,1510,652,265,1005,1417,6945,7145,104.912535,107.933774
22,23.0,Atlanta Hawks,67,16280,2723,6067,0.449,805,2416,0.333,...,2898,1605,523,341,1086,1548,7488,8022,105.192178,112.693864
26,27.0,Cleveland Cavaliers,65,15725,2619,5715,0.458,727,2070,0.351,...,2876,1500,447,211,1073,1188,6948,7461,105.234460,113.004362
23,24.0,Minnesota Timberwolves,64,15560,2586,5865,0.441,853,2540,0.336,...,2868,1525,555,362,981,1370,7248,7523,106.266311,110.298214
16,17.0,Brooklyn Nets,72,17480,2908,6498,0.448,941,2746,0.343,...,3449,1761,464,323,1099,1510,8048,8089,106.967224,107.512161
21,22.0,Orlando Magic,73,17570,2871,6468,0.444,807,2354,0.343,...,3249,1747,599,396,937,1336,7831,7905,107.033514,108.044940
24,25.0,Detroit Pistons,66,15965,2595,5658,0.459,791,2157,0.367,...,2753,1589,490,299,1008,1299,7078,7313,107.041316,110.595245


In [127]:
def assignOffensivePercentile(row):
    ortgs = np.array(df_team['ORTG'])
    return stats.percentileofscore(ortgs, row['ORTG'])

def assignDefensivePercentile(row):
    drtgs = np.array(df_team['DRTG'])
    drtgs *= -1
    
    return stats.percentileofscore(drtgs, row['DRTG'] * -1)

df_team['ORTGP'] = df_team.apply(assignOffensivePercentile, axis=1)
df_team['DRTGP'] = df_team.apply(assignDefensivePercentile, axis=1)

In [128]:
df_team

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,STL,BLK,TOV,PF,PTS,PTSA,ORTG,DRTG,ORTGP,DRTGP
0,1.0,Dallas Mavericks,75,18175,3124,6772,0.461,1136,3095,0.367,...,460,363,953,1462,8776,8405,114.736952,109.886518,100.000000,46.666667
1,2.0,Milwaukee Bucks,73,17595,3160,6638,0.476,1007,2840,0.355,...,526,429,1102,1431,8663,7927,111.507272,102.033724,90.000000,100.000000
2,3.0,Portland Trail Blazers,74,17835,3122,6749,0.463,952,2525,0.377,...,465,451,946,1606,8508,8593,112.038768,113.158103,93.333333,6.666667
3,4.0,Houston Rockets,72,17380,2936,6512,0.451,1126,3261,0.345,...,627,371,1057,1566,8482,8269,111.379573,108.582609,80.000000,56.666667
4,5.0,Los Angeles Clippers,72,17380,2992,6425,0.466,895,2410,0.371,...,511,338,1051,1594,8377,7913,112.192966,105.978625,96.666667,86.666667
5,6.0,New Orleans Pelicans,72,17430,3065,6598,0.465,982,2656,0.370,...,539,358,1181,1524,8341,8434,108.935848,110.150454,46.666667,43.333333
6,7.0,Phoenix Suns,73,17595,3006,6429,0.468,831,2320,0.358,...,559,290,1077,1609,8294,8278,110.734312,110.520694,66.666667,36.666667
7,8.0,Washington Wizards,72,17355,2990,6544,0.457,864,2345,0.368,...,574,309,1020,1634,8238,8574,109.257294,113.713528,50.000000,3.333333
8,9.0,Memphis Grizzlies,73,17570,3106,6634,0.468,798,2297,0.347,...,576,398,1113,1546,8222,8300,107.778623,108.801091,36.666667,53.333333
9,10.0,Boston Celtics,72,17430,2971,6448,0.461,905,2487,0.364,...,595,406,995,1553,8183,7729,111.451609,105.268176,86.666667,90.000000


In [129]:
teamAndWhatTheyAreBadAt = {}
for i,j in df_team.iterrows():
    team = j['Team']
    if team not in teamAndWhatTheyAreBadAt:
        offPercentile = j['ORTGP']
        defPercentile = j['DRTGP']
        if offPercentile > defPercentile:
            teamAndWhatTheyAreBadAt[team] = 'defense'
        else:
            teamAndWhatTheyAreBadAt[team] = 'offense'
            
teamAndWhatTheyAreBadAt

{'Dallas Mavericks': 'defense',
 'Milwaukee Bucks': 'offense',
 'Portland Trail Blazers': 'defense',
 'Houston Rockets': 'defense',
 'Los Angeles Clippers': 'defense',
 'New Orleans Pelicans': 'defense',
 'Phoenix Suns': 'defense',
 'Washington Wizards': 'defense',
 'Memphis Grizzlies': 'offense',
 'Boston Celtics': 'offense',
 'Miami Heat': 'defense',
 'Denver Nuggets': 'defense',
 'Toronto Raptors': 'offense',
 'San Antonio Spurs': 'defense',
 'Philadelphia 76ers': 'offense',
 'Los Angeles Lakers': 'offense',
 'Brooklyn Nets': 'offense',
 'Utah Jazz': 'defense',
 'Indiana Pacers': 'offense',
 'Oklahoma City Thunder': 'offense',
 'Sacramento Kings': 'defense',
 'Orlando Magic': 'offense',
 'Atlanta Hawks': 'defense',
 'Minnesota Timberwolves': 'offense',
 'Detroit Pistons': 'offense',
 'New York Knicks': 'offense',
 'Cleveland Cavaliers': 'defense',
 'Chicago Bulls': 'offense',
 'Golden State Warriors': 'offense',
 'Charlotte Hornets': 'offense'}

In [130]:
for i in teamAndWhatTheyAreBadAt:
    team = i
    badAt = teamAndWhatTheyAreBadAt[i]
    
    # FIND A PLAYER IN THE `team` THAT IS WORSE AT `badAt`
    #    - GET HIS NOT `badAt` data (off or def)
    #    - FIND A PLAYER WHO HAS A SIMILAR NOT `badAt` DATA BUT BETTER `badAt` DATA

In [131]:
# -------------------------
#     K-MEANS CLUSTERING
# -------------------------